In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import re

In [64]:
!mkdir ckpf
check_URL=r'https://s3.eu-west-2.amazonaws.com/suqilu/checkpoint.pth'
vocab_URL=r'https://s3.eu-west-2.amazonaws.com/suqilu/vocab.pth'
!wget $check_URL -o ckpf/checkpoint.pth
!wget $vocab_URL -o ckpf/vocab.pth


mkdir: cannot create directory ‘ckpf’: File exists


In [0]:
class CNNClassifier(nn.Module):
  def __init__(self,weight_matrix_of_word,weight_matrix_of_emoji,num_embeddings, embedding_dim, hidden_dim,target_size,kernel_size):
      super(CNNClassifier, self).__init__()
      self.kernel_size=kernel_size
      self.embedding_word=nn.Embedding.from_pretrained(weight_matrix_of_word)
      self.embedding_emoji=nn.Embedding.from_pretrained(weight_matrix_of_emoji,freeze=False)
      self.conv1=nn.Conv1d(embedding_dim,hidden_dim,kernel_size)
      self.conv2=nn.Conv1d(embedding_dim,hidden_dim,1)
      self.output=nn.Linear(hidden_dim,target_size)
      self.cos=nn.CosineSimilarity(dim=0)
    
  def forward(self,sentence):
    x=self.embedding_word(sentence).view(1,len(sentence),-1).transpose(1,2)
    emoji=self.embedding_emoji.weight
    if(len(sentence)>=self.kernel_size):
      c=self.conv1(x)
      pool1=nn.MaxPool1d(len(sentence)-self.kernel_size+1)
    else:
      c=self.conv2(x)
      pool1=nn.MaxPool1d(len(sentence))
    c=F.relu(c)  
    p=pool1(c).view(1,-1)
    out=self.output(p)
    a=torch.zeros(1,len(emoji))
    for i,j in enumerate(emoji):
      for p,q in enumerate(out):
        a[p][i]=self.cos(q,j)
    return a

In [0]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False

    model.eval()
    return model

model = load_checkpoint('checkpoint.pth')

In [0]:
emotions = ["😊", '😱', "😠", "😢", "😒", "😳", "😣"]

In [0]:
vocab=torch.load("vocab.pth")

In [0]:
def ngram(token,n):
  output=[]
  for i in range(n-1,len(token)):
    ngram=' '.join(token[i-n+1:i+1])
    output.append(ngram)
  return output  

In [0]:
def create_feature(text,nrange=(1,1)):
  text_features=[]
  text=text.lower()
  text_alphanum=re.sub('[^a-z0-9#]',' ',text)
  for n in range(nrange[0],nrange[1]+1):
    text_features += ngram(text_alphanum.split(),n)
  return(text_features)  

In [0]:
def Demo():
  sentence=input("Please type here:")
  sentence=create_feature(sentence)
  b=[]
  for i in sentence:
    try:
      b.append(vocab.index(i))
    except ValueError:
      b
  b=torch.LongTensor(b)    
  output=model(b)
  predict=torch.argmax(output)
  return emotions[predict]

In [0]:
Demo()